In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load datasets
training_data = pd.read_csv("training_data.csv")
target_data = pd.read_csv("target.csv")

# Concatenate the dataframes for later usage (axis=0)
data = pd.concat([training_data, target_data], axis=0, ignore_index=True)


len_texts = [len(x) for x in data['text']]
max(len_texts)

8655

In [2]:

def tokenize(text):
    return text.lower().split()
# label mappings
def build_vocab_and_labels(data):
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    label_to_idx = {"Other": 0}
    sublabel_to_idx = {"Other": 0}
    language_to_idx = {"EN": 0, "PT": 1}

    for _, row in data.iterrows():
        # index the vocab
        for word in tokenize(row["text"]):
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)
        # index the labels
        for label in row["narrative"].split(";"):
            if label not in label_to_idx:
                label_to_idx[label] = len(label_to_idx)
        for sublabel in row["sub_narrative"].split(";"):
            if sublabel not in sublabel_to_idx:
                sublabel_to_idx[sublabel] = len(sublabel_to_idx)

    return word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx


In [3]:

word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx = build_vocab_and_labels(data)
len(word_to_idx)

51779

In [4]:

len(label_to_idx)

22

In [5]:

len(sublabel_to_idx)


93

In [6]:

class DocumentDataset(Dataset):
    def __init__(self, df, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx, max_len=50):
        self.data = df
        self.word_to_idx = word_to_idx
        self.label_to_idx = label_to_idx
        self.sublabel_to_idx = sublabel_to_idx
        self.language_to_idx = language_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens = tokenize(row["text"])
        # assign the unk index if the word isn't indexed already
        input_ids = [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in tokens]
        # make sure the length is always 50 through padding if needed
        input_ids = input_ids[:self.max_len] + [self.word_to_idx["<PAD>"]] * (self.max_len - len(input_ids))

        narrative_labels = [self.label_to_idx[label] for label in row["narrative"].split(";")]
        subnarrative_labels = [self.sublabel_to_idx[label] for label in row["sub_narrative"].split(";")]

        # Multi-label one-hot encoding
        narrative_targets = torch.zeros(len(self.label_to_idx))
        narrative_targets[narrative_labels] = 1

        subnarrative_targets = torch.zeros(len(self.sublabel_to_idx))
        subnarrative_targets[subnarrative_labels] = 1

        return (
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(self.language_to_idx[row["language"]], dtype=torch.long),
            narrative_targets,
            subnarrative_targets
        )



In [7]:

train_dataset = DocumentDataset(training_data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)
val_dataset = DocumentDataset(target_data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [8]:

# LSTM Model with an embedding layer and three classifiers in th end
class LTSMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives):
        super(LTSMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.lang_classifier = nn.Linear(hidden_dim, num_languages)
        self.narrative_classifier = nn.Linear(hidden_dim, num_narratives)
        self.subnarrative_classifier = nn.Linear(hidden_dim, num_subnarratives)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.rnn(embedded)
        hidden = hidden.squeeze(0)

        lang_out = self.lang_classifier(hidden)
        narrative_out = self.narrative_classifier(hidden)
        subnarrative_out = self.subnarrative_classifier(hidden)

        return lang_out, narrative_out, subnarrative_out

In [28]:

vocab_size = len(word_to_idx)
embed_dim = 1024
hidden_dim = 1024
num_languages = len(language_to_idx)
num_narratives = len(label_to_idx)
num_subnarratives = len(sublabel_to_idx)

# the unweighted model to the gpu
model_unweighted = LTSMModel(vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_unweighted.to(device)

# the unweighted loss fucntions
criterion_lang_unweighted = nn.CrossEntropyLoss()
criterion_bce_unweighted = nn.BCEWithLogitsLoss()
optimizer_unweighted = optim.Adam(model_unweighted.parameters(), lr=0.001)
# Calculate the number of parameters
total_params = sum(p.numel() for p in model_unweighted.parameters())
trainable_params = sum(p.numel() for p in model_unweighted.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 61538421
Trainable parameters: 61538421


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Training Loop
num_epochs = 201
for epoch in range(num_epochs):
    model_unweighted.train()
    train_loss = 0
    all_lang_preds, all_lang_targets = [], []
    all_narrative_preds, all_narrative_targets = [], []
    all_subnarrative_preds, all_subnarrative_targets = [], []

    for inputs, lang_targets, narrative_targets, subnarrative_targets in train_loader:
        inputs, lang_targets, narrative_targets, subnarrative_targets = (
            inputs.to(device),
            lang_targets.to(device),
            narrative_targets.to(device),
            subnarrative_targets.to(device),
        )

        optimizer_unweighted.zero_grad()
        # forward pass
        lang_out, narrative_out, subnarrative_out = model_unweighted(inputs)

        # backward pass
        loss_lang = criterion_lang_unweighted(lang_out, lang_targets)
        loss_narrative = criterion_bce_unweighted(narrative_out, narrative_targets)
        loss_subnarrative = criterion_bce_unweighted(subnarrative_out, subnarrative_targets)

        loss = loss_lang + loss_narrative + loss_subnarrative
        loss.backward()
        optimizer_unweighted.step()
        train_loss += loss.item()

        # predictions
        lang_preds = lang_out.argmax(dim=1).cpu().numpy()
        all_lang_preds.extend(lang_preds)
        all_lang_targets.extend(lang_targets.cpu().numpy())

        narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
        all_narrative_preds.extend(narrative_preds)
        all_narrative_targets.extend(narrative_targets.cpu().numpy())

        subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
        all_subnarrative_preds.extend(subnarrative_preds)
        all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

    # Metrics
    lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
    lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

    narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
    narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
    narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

    narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
    narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
    narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

    subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
    subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
    subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

    subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
    subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
    subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

    # Log Epoch Results
    if epoch % 200 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print("\nLanguage Classification Metrics:")
        print(f"  Accuracy: {lang_accuracy:.4f}")
        print(f"  Precision: {lang_precision:.4f}")
        print(f"  Recall: {lang_recall:.4f}")
        print(f"  F1-Score: {lang_f1:.4f}")
        print(f"  Confusion Matrix:\n{lang_cm}")

        print("\nNarrative Classification Metrics (Macro Average):")
        print(f"  Precision: {narrative_macro_precision:.4f}")
        print(f"  Recall: {narrative_macro_recall:.4f}")
        print(f"  F1-Score: {narrative_macro_f1:.4f}")

        print("\nNarrative Classification Metrics (Micro Average):")
        print(f"  Precision: {narrative_micro_precision:.4f}")
        print(f"  Recall: {narrative_micro_recall:.4f}")
        print(f"  F1-Score: {narrative_micro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Macro Average):")
        print(f"  Precision: {subnarrative_macro_precision:.4f}")
        print(f"  Recall: {subnarrative_macro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Micro Average):")
        print(f"  Precision: {subnarrative_micro_precision:.4f}")
        print(f"  Recall: {subnarrative_micro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_micro_f1:.4f}")

        print("=" * 50)


Epoch 1/201
Train Loss: 1.1416

Language Classification Metrics:
  Accuracy: 0.8773
  Precision: 0.8774
  Recall: 0.8773
  F1-Score: 0.8773
  Confusion Matrix:
[[352  47]
 [ 51 349]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.1106
  Recall: 0.0838
  F1-Score: 0.0854

Narrative Classification Metrics (Micro Average):
  Precision: 0.1313
  Recall: 0.1044
  F1-Score: 0.1163

Sub-Narrative Classification Metrics (Macro Average):
  Precision: 0.0316
  Recall: 0.0825
  F1-Score: 0.0382

Sub-Narrative Classification Metrics (Micro Average):
  Precision: 0.0364
  Recall: 0.0989
  F1-Score: 0.0532
Epoch 201/201
Train Loss: 0.0029

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[399   0]
 [  0 400]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.9865
  Recall: 0.9887
  F1-Score: 0.9876

Narrative Classification Metrics (Micro Average):
  Precision: 0.9891
  Recall: 0.9923
 

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Validation Loop
train_loss = 0
all_lang_preds, all_lang_targets = [], []
all_narrative_preds, all_narrative_targets = [], []
all_subnarrative_preds, all_subnarrative_targets = [], []

for inputs, lang_targets, narrative_targets, subnarrative_targets in val_loader:
    inputs, lang_targets, narrative_targets, subnarrative_targets = (
        inputs.to(device),
        lang_targets.to(device),
        narrative_targets.to(device),
        subnarrative_targets.to(device),
    )
    # forward pass
    lang_out, narrative_out, subnarrative_out = model_unweighted(inputs)

    # Losses
    loss_lang = criterion_lang_unweighted(lang_out, lang_targets)
    loss_narrative = criterion_bce_unweighted(narrative_out, narrative_targets)
    loss_subnarrative = criterion_bce_unweighted(subnarrative_out, subnarrative_targets)

    loss = loss_lang + loss_narrative + loss_subnarrative
    train_loss += loss.item()

    # predictions
    lang_preds = lang_out.argmax(dim=1).cpu().numpy()
    all_lang_preds.extend(lang_preds)
    all_lang_targets.extend(lang_targets.cpu().numpy())

    narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
    all_narrative_preds.extend(narrative_preds)
    all_narrative_targets.extend(narrative_targets.cpu().numpy())

    subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
    all_subnarrative_preds.extend(subnarrative_preds)
    all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

#  Metrics
lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

# Narrative Metrics
narrative_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

narrative_micro_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

# Sub-Narrative Metrics
subnarrative_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

subnarrative_micro_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

# Log Epoch Results
print(f"Validation Loss: {train_loss / len(val_loader):.4f}")
print("\nLanguage Classification Metrics:")
print(f"  Accuracy: {lang_accuracy:.4f}")
print(f"  Precision: {lang_precision:.4f}")
print(f"  Recall: {lang_recall:.4f}")
print(f"  F1-Score: {lang_f1:.4f}")
print(f"  Confusion Matrix:\n{lang_cm}")

print("\nNarrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {narrative_accuracy:.4f}")
print(f"  Precision: {narrative_macro_precision:.4f}")
print(f"  Recall: {narrative_macro_recall:.4f}")
print(f"  F1-Score: {narrative_macro_f1:.4f}")

print("\nNarrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {narrative_micro_accuracy:.4f}")
print(f"  Precision: {narrative_micro_precision:.4f}")
print(f"  Recall: {narrative_micro_recall:.4f}")
print(f"  F1-Score: {narrative_micro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {subnarrative_accuracy:.4f}")
print(f"  Precision: {subnarrative_macro_precision:.4f}")
print(f"  Recall: {subnarrative_macro_recall:.4f}")
print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {subnarrative_micro_accuracy:.4f}")
print(f"  Precision: {subnarrative_micro_precision:.4f}")
print(f"  Recall: {subnarrative_micro_recall:.4f}")
print(f"  F1-Score: {subnarrative_micro_f1:.4f}")
print("=" * 50)



Validation Loss: 1.1619

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[41  0]
 [ 0 35]]

Narrative Classification Metrics (Macro Average):
  Accuracy: 0.1316
  Precision: 0.2203
  Recall: 0.1121
  F1-Score: 0.1279

Narrative Classification Metrics (Micro Average):
  Accuracy: 0.1316
  Precision: 0.3472
  Recall: 0.1701
  F1-Score: 0.2283

Sub-Narrative Classification Metrics (Macro Average):
  Accuracy: 0.0921
  Precision: 0.0397
  Recall: 0.0258
  F1-Score: 0.0249

Sub-Narrative Classification Metrics (Micro Average):
  Accuracy: 0.0921
  Precision: 0.2800
  Recall: 0.0725
  F1-Score: 0.1152


In [31]:
import torch

all_dataset = DocumentDataset(data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)


# get the labels
narrative_targets = torch.stack([torch.tensor(target[2]) for target in all_dataset]).to(device)
subnarrative_targets = torch.stack([torch.tensor(target[3]) for target in all_dataset]).to(device)

# Compute the class weights for both narrative and sub_narrative
narrative_class_weights = narrative_targets.sum(dim=0)  # Sum along rows (class-wise sum)
subnarrative_class_weights = subnarrative_targets.sum(dim=0)  # Sum along rows (class-wise sum)

# Normalize the class weights by dividing by the maximum class weight
pos_weight_narrative = (narrative_class_weights.max() / narrative_class_weights).to(device)
pos_weight_subnarrative = (subnarrative_class_weights.max() / subnarrative_class_weights).to(device)


print("Narrative Class Weights:", narrative_class_weights)
print("Subnarrative Class Weights:", subnarrative_class_weights)
print("Pos Weight for Narrative:", pos_weight_narrative)
print("Pos Weight for Subnarrative:", pos_weight_subnarrative)

# update the loss functions
criterion_narrative_weighted = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight_narrative)
criterion_subnarrative_weighted = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight_subnarrative)



/tmp/ipykernel_10690/1170623561.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  narrative_targets = torch.stack([torch.tensor(target[2]) for target in all_dataset]).to(device)
/tmp/ipykernel_10690/1170623561.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  subnarrative_targets = torch.stack([torch.tensor(target[3]) for target in all_dataset]).to(device)


Narrative Class Weights: tensor([208.,  63.,  26.,  40., 129., 117.,  98.,  80., 171.,  14., 182.,  35.,
         20., 138.,  25.,  52.,  25.,  44.,   8.,  78., 148.,   7.],
       device='cuda:0')
Subnarrative Class Weights: tensor([208.,  23.,  16.,  29.,  30.,   6.,  24.,  27.,  19.,  51.,  24.,  78.,
         14.,  58.,  43.,  25.,  11., 127.,  38.,  35.,   1.,   2.,  13.,  10.,
         42.,  30.,  42.,  19.,   3.,  21.,  25.,  15.,  73.,  13.,   9.,  13.,
         11.,  11.,  27.,  18.,  51.,   2.,  76.,   6.,   5.,   5.,  14.,  31.,
         38.,  35.,  10.,  18.,  11.,   3.,   6.,  82.,  31.,  15.,  13.,  15.,
          8.,   3.,  32.,  25.,   3.,   4.,  24.,  20.,  18.,  28.,   1.,  38.,
          2.,  69.,   9.,   3.,   2.,   1.,   5.,   8.,   2.,   3.,  28., 106.,
         11.,   4.,  67.,   4.,   1.,   2.,   1.,   1.,   1.], device='cuda:0')
Pos Weight for Narrative: tensor([ 1.0000,  3.3016,  8.0000,  5.2000,  1.6124,  1.7778,  2.1224,  2.6000,
         1.2164, 14.8571,  1

In [32]:
# weighted model
model_weighted = LTSMModel(vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weighted.to(device)

criterion_lang_weighted = nn.CrossEntropyLoss()
optimizer_weighted = optim.Adam(model_weighted.parameters(), lr=0.001)

# Training Loop
num_epochs = 201
for epoch in range(num_epochs):
    model_weighted.train()
    train_loss = 0
    all_lang_preds, all_lang_targets = [], []
    all_narrative_preds, all_narrative_targets = [], []
    all_subnarrative_preds, all_subnarrative_targets = [], []

    for inputs, lang_targets, narrative_targets, subnarrative_targets in train_loader:
        inputs, lang_targets, narrative_targets, subnarrative_targets = (
            inputs.to(device),
            lang_targets.to(device),
            narrative_targets.to(device),
            subnarrative_targets.to(device),
        )

        optimizer_weighted.zero_grad()
        # forward pass
        lang_out, narrative_out, subnarrative_out = model_weighted(inputs)

        # backward pass
        loss_lang = criterion_lang_weighted(lang_out, lang_targets)
        loss_narrative = criterion_narrative_weighted(narrative_out, narrative_targets)
        loss_subnarrative = criterion_subnarrative_weighted(subnarrative_out, subnarrative_targets)

        loss = loss_lang + loss_narrative + loss_subnarrative
        loss.backward()
        optimizer_weighted.step()
        train_loss += loss.item()

        # predictions
        lang_preds = lang_out.argmax(dim=1).cpu().numpy()
        all_lang_preds.extend(lang_preds)
        all_lang_targets.extend(lang_targets.cpu().numpy())

        narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
        all_narrative_preds.extend(narrative_preds)
        all_narrative_targets.extend(narrative_targets.cpu().numpy())

        subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
        all_subnarrative_preds.extend(subnarrative_preds)
        all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

    #  Metrics
    lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
    lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

    # Compute Metrics for Narrative (Multi-Label) Classification
    narrative_precision = precision_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)
    narrative_recall = recall_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)
    narrative_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)

    # Compute Metrics for Sub-Narrative (Multi-Label) Classification
    subnarrative_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)
    subnarrative_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)
    subnarrative_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)

    # Log Epoch Results
    if epoch % 200 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print("\nLanguage Classification Metrics:")
        print(f"  Accuracy: {lang_accuracy:.4f}")
        print(f"  Precision: {lang_precision:.4f}")
        print(f"  Recall: {lang_recall:.4f}")
        print(f"  F1-Score: {lang_f1:.4f}")
        print(f"  Confusion Matrix:\n{lang_cm}")

        print("\nNarrative Classification Metrics (Macro Average):")
        print(f"  Precision: {narrative_macro_precision:.4f}")
        print(f"  Recall: {narrative_macro_recall:.4f}")
        print(f"  F1-Score: {narrative_macro_f1:.4f}")

        print("\nNarrative Classification Metrics (Micro Average):")
        print(f"  Precision: {narrative_micro_precision:.4f}")
        print(f"  Recall: {narrative_micro_recall:.4f}")
        print(f"  F1-Score: {narrative_micro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Macro Average):")
        print(f"  Precision: {subnarrative_macro_precision:.4f}")
        print(f"  Recall: {subnarrative_macro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Micro Average):")
        print(f"  Precision: {subnarrative_micro_precision:.4f}")
        print(f"  Recall: {subnarrative_micro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_micro_f1:.4f}")

        print("=" * 50)


Epoch 1/201
Train Loss: 1.4879

Language Classification Metrics:
  Accuracy: 0.9537
  Precision: 0.9538
  Recall: 0.9537
  F1-Score: 0.9537
  Confusion Matrix:
[[384  15]
 [ 22 378]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.2203
  Recall: 0.1121
  F1-Score: 0.1279

Narrative Classification Metrics (Micro Average):
  Precision: 0.3472
  Recall: 0.1701
  F1-Score: 0.2283

Sub-Narrative Classification Metrics (Macro Average):
  Precision: 0.0397
  Recall: 0.0258
  F1-Score: 0.0249

Sub-Narrative Classification Metrics (Micro Average):
  Precision: 0.2800
  Recall: 0.0725
  F1-Score: 0.1152
Epoch 201/201
Train Loss: 0.0058

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[399   0]
 [  0 400]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.2203
  Recall: 0.1121
  F1-Score: 0.1279

Narrative Classification Metrics (Micro Average):
  Precision: 0.3472
  Recall: 0.1701
 

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Validation Loop
train_loss = 0
all_lang_preds, all_lang_targets = [], []
all_narrative_preds, all_narrative_targets = [], []
all_subnarrative_preds, all_subnarrative_targets = [], []

for inputs, lang_targets, narrative_targets, subnarrative_targets in val_loader:
    inputs, lang_targets, narrative_targets, subnarrative_targets = (
        inputs.to(device),
        lang_targets.to(device),
        narrative_targets.to(device),
        subnarrative_targets.to(device),
    )
    # forward pass
    lang_out, narrative_out, subnarrative_out = model_weighted(inputs)

    # backward pass
    loss_lang = criterion_lang_weighted(lang_out, lang_targets)
    loss_narrative = criterion_narrative_weighted(narrative_out, narrative_targets)
    loss_subnarrative = criterion_subnarrative_weighted(subnarrative_out, subnarrative_targets)

    loss = loss_lang + loss_narrative + loss_subnarrative
    train_loss += loss.item()

    # predictions
    lang_preds = lang_out.argmax(dim=1).cpu().numpy()
    all_lang_preds.extend(lang_preds)
    all_lang_targets.extend(lang_targets.cpu().numpy())

    narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
    all_narrative_preds.extend(narrative_preds)
    all_narrative_targets.extend(narrative_targets.cpu().numpy())

    subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
    all_subnarrative_preds.extend(subnarrative_preds)
    all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

#  Metrics
lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

# Narrative Metrics
narrative_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

narrative_micro_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

# Sub-Narrative Metrics
subnarrative_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

subnarrative_micro_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

# Log Epoch Results
print(f"Validation Loss: {train_loss / len(val_loader):.4f}")
print("\nLanguage Classification Metrics:")
print(f"  Accuracy: {lang_accuracy:.4f}")
print(f"  Precision: {lang_precision:.4f}")
print(f"  Recall: {lang_recall:.4f}")
print(f"  F1-Score: {lang_f1:.4f}")
print(f"  Confusion Matrix:\n{lang_cm}")

print("\nNarrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {narrative_accuracy:.4f}")
print(f"  Precision: {narrative_macro_precision:.4f}")
print(f"  Recall: {narrative_macro_recall:.4f}")
print(f"  F1-Score: {narrative_macro_f1:.4f}")

print("\nNarrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {narrative_micro_accuracy:.4f}")
print(f"  Precision: {narrative_micro_precision:.4f}")
print(f"  Recall: {narrative_micro_recall:.4f}")
print(f"  F1-Score: {narrative_micro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {subnarrative_accuracy:.4f}")
print(f"  Precision: {subnarrative_macro_precision:.4f}")
print(f"  Recall: {subnarrative_macro_recall:.4f}")
print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {subnarrative_micro_accuracy:.4f}")
print(f"  Precision: {subnarrative_micro_precision:.4f}")
print(f"  Recall: {subnarrative_micro_recall:.4f}")
print(f"  F1-Score: {subnarrative_micro_f1:.4f}")
print("=" * 50)


Validation Loss: 5.8921

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[41  0]
 [ 0 35]]

Narrative Classification Metrics (Macro Average):
  Accuracy: 0.1184
  Precision: 0.1879
  Recall: 0.0955
  F1-Score: 0.1155

Narrative Classification Metrics (Micro Average):
  Accuracy: 0.1184
  Precision: 0.4068
  Recall: 0.1633
  F1-Score: 0.2330

Sub-Narrative Classification Metrics (Macro Average):
  Accuracy: 0.0789
  Precision: 0.0497
  Recall: 0.0269
  F1-Score: 0.0288

Sub-Narrative Classification Metrics (Micro Average):
  Accuracy: 0.0789
  Precision: 0.3636
  Recall: 0.0829
  F1-Score: 0.1350
